<h1 align="center">Organization Info</h1> 

* Дедлайн **DD MM 2018 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 594_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall_<номер_группы>_<фамилия>``, к примеру -- ``ML2018_fall_495_ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb, к примеру`` -- ``ivanov_401_task7.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall Question <Содержание вопроса>``


--------
- **PS1:** Используются автоматические фильтры, и просто не найдем ваше дз, если вы неаккуратно его подпишите.
- **PS2:**  Просроченный дедлайн снижает максимальный вес задания по формуле, указнной на первом семинаре
- **PS3:** Допустимы исправление кода предложенного кода ниже, если вы считаете

<h1 align="center">Checking Questions</h1> 

**Вопрос 1**: Чем LSTM лучше/хуже чем обычная RNN?

В LSTM состояние С, которое играет роль памяти,поэтому она эффективнее, чем обычная RNN. Но появляется много весов, значит учить надо дольше и нужно больше данных.

**Вопрос 2**:  Выпишите производную $\frac{d c_{n+1}}{d c_{k}}$ для LSTM http://colah.github.io/posts/2015-08-Understanding-LSTMs/, объясните формулу, когда производная затухает, когда взрывается?

$\frac{d c_{n+1}}{d c_{n}} = f_{n+1} = \sigma(W_{f}[h_{n+1}, x_{n+1}] + b_{f})$э

Так как есть сигмоида, то градиент никогда не превышает 1, следовательно он не взрыватеся.

**Вопрос 3**: Зачем нужен TBPTT почему BPTT плох?

Так как могут использоваться последовательности произвольной длины то на BPTT нужно протаскивать производные через произвольное количество слоев, что ослождняет 
вычисления и способствует затуханию. При TBPTT эффекты от этого меньше.


**Вопрос 4**: Как комбинировать рекуррентные и сверточные сети, а главное зачем? Приведите несколько примеров реальных задач.

Можно один из последних сверточной сети подать на вход рекуррентной сети. Так можно генерировать, к примеру, описания по изображению и распозновать текст на документе.

**Вопрос 5**: Можно ли использовать сверточные сети для классификации текстов? Если нет обоснуйте :D, если да то как? как решить проблему с произвольной длинной входа?

При использовании фильтров с ядрами вида формы $[n,1]$ получим n-грамм анализ текста. На после convolution слоев поствить RNN, которая на вход принимает выход сверточных слоев и распределение на прошлой итерации. На первом этапе подать туда равномерное распределение.

**Вопрос 6**: Attention - что это такое, где применяют и как? Приведите пример использования на какой-нибудь задаче

Аttention - механизм, позволяющий использовать hidden-state предыдущих слоев в последущих слоях. На вход подается произведение hidden-state с предыдущего слоя с некоторым набором весов. К примеру, при машинном переводе attention позволяет акцентировать внимание на некоторых частях исходного текста.

## Grading
* starting at zero points
* +2 for describing your iteration path in a report below (compare models).
* +2 for correct check questions
* +3 (7 total) for 99% accuracy with simple NMT model on __TEST__ dataset
* +3 (10 total) for 99% accuracy with attention NMT model on __TEST__ dataset
----
* tatoeba bonus for accuracy on __TEST__ dataset:
    * +2 for report
    * 60% (14 total)
    * 65% (16 total)
    * 70% (18 total)
    * 75% (20 total)
    
## Bonus points

Common ways to get bonus points are:
* Get higher score, obviously.
* Anything special about your NN. For example "A super-small/fast NN that gets 99%" gets a bonus.
* Any detailed analysis of the results. (attention maps, whatever)

---

In [0]:
# additional packages for this notebook

In [5]:
! pip install faker tqdm babel

    100% |████████████████████████████████| 747kB 6.4MB/s 
    100% |████████████████████████████████| 51kB 12.4MB/s 
    100% |████████████████████████████████| 6.8MB 5.6MB/s 
    100% |████████████████████████████████| 81kB 28.8MB/s 


## Task - translation

The machine translation is old and well-known field in natural language processing. From the 1950s scientists tried to create a model to automatically translate from say French to English. Nowadays it became possible and the attention mechanism takes great part in that. Here the example image with attention map for the neural machine translation of sample phrase:
<p align="center">
  <img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.23.48-PM.png" width="400">
</p>

In our lab we will concentrate on much simplier task: we will translate from human readable date to machine readable one.

1.   List item
2.   List item



To do this we need to get one more concept - Sequence-to-Sequence language modeling.
The idea of such architecture is here:
<p aling="center">
<img src="./img/simple_nmt.jpg" width="400">
</p>

There is an Embeding layer at the bottom, the RNN in the middle and softmax as an output.

In [0]:
from keras.layers import Embedding, Bidirectional
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.layers import TimeDistributed

import keras.backend as K
import numpy as np

---

### Data

Now we need to generate data. It will be dates in different text formats and in fixed output format.

In [0]:
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import numpy as np

In [0]:
fake = Faker()

FORMATS = ['short',
           'medium',
           'long',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'dd MMM YYY',
           'd MMM, YYY',
           'd MMMM, YYY',
           'dd, MMM YYY',
           'd MM YY',
           'd MMMM YYY',
           'MMMM d YYY',
           'MMMM d, YYY',
           'dd.MM.YY']

# change this if you want it to work with another language
LOCALES = ['en_US']

In [0]:
def create_date():
    """
        Creates some fake dates 
        :returns: tuple containing human readable string, machine readable string, and date object
    """
    dt = fake.date_object()

    try:
        human_readable = format_date(dt, format=random.choice(FORMATS), locale=random.choice(LOCALES))

        case_change = random.choice([0,1,2])
        if case_change == 1:
            human_readable = human_readable.upper()
        elif case_change == 2:
            human_readable = human_readable.lower()
        # if case_change == 0, do nothing

        machine_readable = dt.isoformat()
    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, dt

In [0]:
def create_dataset(n_examples):
    """
        Creates a dataset with n_examples and vocabularies
        :n_examples: the number of examples to generate
    """
    human_vocab = set()
    machine_vocab = set()
    dataset = []

    for i in tqdm(range(n_examples)):
        h, m, _ = create_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))

    human = dict(zip(list(human_vocab) + ['<unk>', '<pad>'], 
                     list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(list(machine_vocab) + ['<unk>', '<pad>']))
    machine = {v:k for k,v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

In [0]:
def string_to_int(string, lenght, vocab):
    if len(string) > lenght:
        string = string[:lenght]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < lenght:
        rep += [vocab['<pad>']] * (lenght - len(string))
    
    return rep

In [0]:
def int_to_string(ints, inv_vocab):
    return [inv_vocab[i] for i in ints]

Actually generating data:

In [13]:
fake.seed(42)
random.seed(42)
N = int(3e5)
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(N)

 58%|█████▊    | 174723/300000 [00:09<00:07, 17546.87it/s]/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 300000/300000 [00:17<00:00, 17405.68it/s]


In [14]:
#@title Default title text
dataset[2]

('tuesday, september 14, 1971', '1971-09-14')

In [0]:
TIME_STEPS = 20 # change me if u want

In [0]:
inputs, targets = zip(*dataset)
inputs = np.array([string_to_int(i, TIME_STEPS, human_vocab) for i in inputs])
targets = [string_to_int(t, TIME_STEPS, machine_vocab) for t in targets]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), targets)))

In [0]:
X_train, y_train, X_valid, y_valid, X_test, y_test = (
    inputs[:int(2e5)], targets[:int(2e5)], 
    inputs[int(2e5):-int(5e4)], targets[int(2e5):-int(5e4)],  
    inputs[-int(5e4):], targets[-int(5e4):], )

---

### Part 1: Simple NMT

In [0]:
# :good-enouht:
ENCODER_UNITS = 32 # change me if u want
DECODER_UNITS = 32 # change me if u want
TIME_STEPS = 20 # change me if u want

In [0]:
# input - [bs; in_time_len]
# output - [bs; out_time_len]; out_time_len=10

def model_simple_nmt(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))

    embedding = Embedding(in_chars, in_chars, input_length=TIME_STEPS)(inputs) # перевод данных в вектора
    lstm1 = LSTM(ENCODER_UNITS, return_sequences=False)(embedding) # encoder
    repeat1 = RepeatVector(TIME_STEPS)(lstm1) # повторяем выход первого слоя, чтобы было что подавать на второй

    lstm2 = LSTM(DECODER_UNITS, return_sequences=True, input_shape=(ENCODER_UNITS, TIME_STEPS))(repeat1) # decoder
    dense1 = Dense(out_chars)(lstm2) # полносвязный слой и активациооная функция для получения ответа
    output = Activation('softmax')(dense1)


    model = Model(input=[inputs], output=output)
    return model

In [49]:
m1 = model_simple_nmt(len(human_vocab), len(machine_vocab))

m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 20, 60)            3600      
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                11904     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 20, 32)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 20, 32)            8320      
_________________________________________________________________
dense_8 (Dense)              (None, 20, 13)            429       
_________________________________________________________________
activation_5 (Activation)    (None, 20, 13)            0         
Total para

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


In [50]:
m1.fit(
    [X_train], y_train, 
    validation_data=(X_valid, y_valid),
    epochs=30, batch_size=1000, 
    validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/30
200000/200000 [==============================] - 25s 123us/step - loss: 1.3604 - acc: 0.5745 - val_loss: 1.0532 - val_acc: 0.6415
Epoch 2/30
200000/200000 [==============================] - 23s 114us/step - loss: 1.0120 - acc: 0.6488 - val_loss: 0.9769 - val_acc: 0.6514
Epoch 3/30
 29000/200000 [===>..........................] - ETA: 18s - loss: 0.9728 - acc: 0.6514

200000/200000 [==============================] - 23s 114us/step - loss: 0.9502 - acc: 0.6753 - val_loss: 0.9145 - val_acc: 0.7048
Epoch 4/30
200000/200000 [==============================] - 23s 114us/step - loss: 0.8698 - acc: 0.7167 - val_loss: 0.8255 - val_acc: 0.7248
Epoch 5/30
108000/200000 [===============>..............] - ETA: 9s - loss: 0.8016 - acc: 0.7393

200000/200000 [==============================] - 23s 114us/step - loss: 0.7761 - acc: 0.7561 - val_loss: 0.7163 - val_acc: 0.7831
Epoch 6/30
200000/200000 [==============================] - 23s 114us/step - loss: 0.6452 - acc: 0.7887 - val_loss: 0.5734 - val_acc: 0.7953
Epoch 7/30
126000/200000 [=================>............] - ETA: 8s - loss: 0.5345 - acc: 0.8044

200000/200000 [==============================] - 23s 117us/step - loss: 0.5143 - acc: 0.8112 - val_loss: 0.4644 - val_acc: 0.8280
Epoch 8/30
200000/200000 [==============================] - 23s 115us/step - loss: 0.4320 - acc: 0.8401 - val_loss: 0.4020 - val_acc: 0.8496
Epoch 9/30
131000/200000 [==================>...........] - ETA: 7s - loss: 0.3841 - acc: 0.8552

200000/200000 [==============================] - 23s 115us/step - loss: 0.3768 - acc: 0.8577 - val_loss: 0.3559 - val_acc: 0.8643
Epoch 10/30
200000/200000 [==============================] - 23s 115us/step - loss: 0.3386 - acc: 0.8717 - val_loss: 0.3238 - val_acc: 0.8751
Epoch 11/30
131000/200000 [==================>...........] - ETA: 7s - loss: 0.3122 - acc: 0.8814

...

200000/200000 [==============================] - 24s 118us/step - loss: 0.1031 - acc: 0.9716 - val_loss: 0.0968 - val_acc: 0.9735
Epoch 22/30
200000/200000 [==============================] - 24s 118us/step - loss: 0.0916 - acc: 0.9751 - val_loss: 0.0859 - val_acc: 0.9768
Epoch 23/30
130000/200000 [==================>...........] - ETA: 7s - loss: 0.0827 - acc: 0.9779

200000/200000 [==============================] - 24s 118us/step - loss: 0.0811 - acc: 0.9783 - val_loss: 0.0765 - val_acc: 0.9795
Epoch 24/30
200000/200000 [==============================] - 24s 118us/step - loss: 0.0710 - acc: 0.9819 - val_loss: 0.0663 - val_acc: 0.9843
Epoch 25/30
130000/200000 [==================>...........] - ETA: 7s - loss: 0.0637 - acc: 0.9852

200000/200000 [==============================] - 24s 118us/step - loss: 0.0619 - acc: 0.9859 - val_loss: 0.0568 - val_acc: 0.9878
Epoch 26/30
200000/200000 [==============================] - 23s 117us/step - loss: 0.0536 - acc: 0.9883 - val_loss: 0.0498 - val_acc: 0.9891
Epoch 27/30
130000/200000 [==================>...........] - ETA: 7s - loss: 0.0483 - acc: 0.9895

200000/200000 [==============================] - 23s 115us/step - loss: 0.0474 - acc: 0.9896 - val_loss: 0.0464 - val_acc: 0.9894
Epoch 28/30
200000/200000 [==============================] - 23s 114us/step - loss: 0.0428 - acc: 0.9904 - val_loss: 0.0400 - val_acc: 0.9909
Epoch 29/30
130000/200000 [==================>...........] - ETA: 7s - loss: 0.0396 - acc: 0.9908

200000/200000 [==============================] - 23s 114us/step - loss: 0.0388 - acc: 0.9910 - val_loss: 0.0368 - val_acc: 0.9913
Epoch 30/30
200000/200000 [==============================] - 23s 114us/step - loss: 0.0355 - acc: 0.9915 - val_loss: 0.0340 - val_acc: 0.9917


In [51]:
m1.evaluate([X_test], y_test)

50000/50000 [==============================] - 43s 864us/step


[0.034002687017917636, 0.9916500009727478]

Lets check our model:

In [0]:
EXAMPLES = ['3 May 1979', '5 Apr 09', '20th February 2016', 'Wed 10 Jul 2007']

def run_example(model, input_vocabulary, inv_output_vocabulary, text):
    encoded = string_to_int(text, TIME_STEPS, input_vocabulary)
    prediction = model.predict(np.array([encoded]))
    prediction = np.argmax(prediction[0], axis=-1)
    return int_to_string(prediction, inv_output_vocabulary)

def run_examples(model, input_vocabulary, inv_output_vocabulary, examples=EXAMPLES):
    predicted = []
    for example in examples:
        predicted.append(''.join(run_example(model, input_vocabulary, inv_output_vocabulary, example)))
        print('input:', example)
        print('output:', predicted[-1])
    return predicted

In [53]:
run_examples(m1, human_vocab, inv_machine_vocab)

input: 3 May 1979
output: 1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 5 Apr 09
output: 2009-06-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: 20th February 2016
output: 2016-06-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
input: Wed 10 Jul 2007
output: 2007-07-06<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


['1979-05-03<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2009-06-05<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2016-06-20<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '2007-07-06<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [0]:
---

### Part 2: All u need is attention

Here we use more complex idea that simple seq2seq: we're adding two explicit parts of our network - encoder and decoder (which is applied attention on). The explanatory picture for this idea is below:
<p aling="center"><img src="https://i.stack.imgur.com/Zwsmz.png"></p>

The lower part of the network is encoding the input to some hidden intermediate representation and the upper part is decoing the hidвen represenataion into some readable output.

In [0]:
# :good-enouht:
ENCODER_UNITS = 32 # change me if u want
DECODER_UNITS = 32 # change me if u want
TIME_STEPS = 20 # change me if u want

In [0]:
from keras.layers import merge



In [0]:
def model_not_so_simple_nmt(in_chars, out_chars):
    # RNN encoder -> hidden representation -> RNN decoder
    inputs = Input(shape=(TIME_STEPS,))
    embedding = Embedding(in_chars, in_chars, input_length=TIME_STEPS)(inputs) # перевод данных в вектора
    lstm1 = LSTM(ENCODER_UNITS, return_sequences=True)(embedding)  # encoder
    
    #генерируем матрицу весов и мерджим её со входом предыдущего слоя
    input_dim = int(lstm1.shape[2])
    a = Permute((2, 1))(lstm1)
    a = Reshape((input_dim, TIME_STEPS))(a) 
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    result = merge([lstm1, a_probs], name='attention_mul', mode='mul')

    lstm2 = LSTM(DECODER_UNITS, return_sequences=True, input_shape=(ENCODER_UNITS, TIME_STEPS))(result)# decoder
    dense1 = Dense(out_chars)(lstm2)# полносвязный слой и активациооная функция для получения ответа
    output = Activation('softmax')(dense1)


    model = Model(input=[inputs], output=output)
    return model

In [31]:
m = model_not_so_simple_nmt(len(human_vocab), len(machine_vocab))

m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 20, 60)       3600        input_4[0][0]                    
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 20, 32)       11904       embedding_4[0][0]                
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 32, 20)       0           lstm_7[0][0]                     
__________________________________________________________________________________________________
reshape_3 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


In [0]:
# 133 эпох с batch size 1000

In [44]:
m.fit(
    [X_train], y_train, 
    validation_data=(X_valid, y_valid),
    epochs=1, batch_size=1000, 
    validation_split=0.1)

Train on 200000 samples, validate on 50000 samples
Epoch 1/1
200000/200000 [==============================] - 24s 118us/step - loss: 0.0231 - acc: 0.9913 - val_loss: 0.0229 - val_acc: 0.9913


In [45]:
m.evaluate([X_test], y_test)

50000/50000 [==============================] - 44s 874us/step


[0.022750731980353593, 0.9913490004730224]

### Report

* final architectures

1) Простая сеть 
- embedding  - перевод входа в вектор
- LSTM - кодирование входа
- repeatLayer - размножение последнего выхода для передачи в декодер
- LSTM - декодирование
- dence + activation - получение результата

2) Сеть с вниманием

- embedding  - перевод входа в вектор
- LSTM - кодирование входа

- attention generation block :
- - permute - меняем размерность входа
- - reshape - нужен, чтобы убедиться, что размерность поменялась правильно
- - dence  + softmax - генерируем некоторые числа и переводим их в веса от 0 до 1
- - multiply - применяем веса ко входу

- LSTM - декодирование
- dence + activation - получение результата




* comparison

Вторая сеть учится дольше до нужного результата так как в ней просто больше весов.
* as well as training method and tricks


---

## Part 3*: tatoeba - real NMT

### Data

In [0]:
# dataset from http://www.manythings.org/anki/

In [54]:
! wget http://www.manythings.org/anki/rus-eng.zip

--2018-05-05 09:02:53--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2400:cb00:2048:1::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6366669 (6.1M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]   6.07M  14.0MB/s    in 0.4s    

2018-05-05 09:02:53 (14.0 MB/s) - ‘rus-eng.zip’ saved [6366669/6366669]



In [55]:
! unzip ./rus-eng.zip

Archive:  ./rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [0]:
with open("./rus.txt") as fin:
    data = fin.readlines()
data = list(map(lambda x: x.replace("\n", "").lower(), data))

In [57]:
len(data)

300108

In [0]:
data = data[:int(1e5)]

In [59]:
len(data)

100000

In [60]:
data[-1]

'tom is here to see you.\tк тебе том пришёл.'

----

In [0]:
source = list(map(lambda x: x.split("\t")[0], data))
target = list(map(lambda x: x.split("\t")[1], data))

In [0]:
source_vocab = set("".join(source).strip())
target_vocab = set("".join(target).strip())

In [0]:
source_vocab = dict(zip(
    list(source_vocab) + ['<unk>', '<pad>'], 
    list(range(len(source_vocab) + 2))))
target_vocab = dict(zip(
    list(target_vocab) + ['<unk>', '<pad>'], 
    list(range(len(target_vocab) + 2))))
inv_target_vocab = dict(enumerate(list(target_vocab) + ['<unk>', '<pad>']))

In [0]:
TIME_STEPS = 32
ENCODER_UNITS = 256
DECODER_UNITS = 256

In [0]:
def model_simple_nmt_tatoeba(in_chars, out_chars):
    inputs = Input(shape=(TIME_STEPS,))
    embedding = Embedding(in_chars, in_chars, input_length=TIME_STEPS)(inputs) # перевод данных в вектора
    lstm1 = LSTM(ENCODER_UNITS, return_sequences=True)(embedding)  # encoder
    
    #генерируем матрицу весов и мерджим её со входом предыдущего слоя
    input_dim = int(lstm1.shape[2])
    a = Permute((2, 1))(lstm1)
    a = Reshape((input_dim, TIME_STEPS))(a) 
    a = Dense(TIME_STEPS, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    result = merge([lstm1, a_probs], name='attention_mul', mode='mul')

    lstm2 = LSTM(DECODER_UNITS, return_sequences=True, input_shape=(ENCODER_UNITS, TIME_STEPS))(result)# decoder
    dense1 = Dense(out_chars)(lstm2)# полносвязный слой и активациооная функция для получения ответа
    output = Activation('softmax')(dense1)


    model = Model(input=[inputs], output=output)
    return model

In [77]:
m_t = model_simple_nmt_tatoeba(len(source_vocab), len(target_vocab))

m_t.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(m_t.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 32, 54)       2916        input_11[0][0]                   
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (None, 32, 256)      318464      embedding_11[0][0]               
__________________________________________________________________________________________________
permute_6 (Permute)             (None, 256, 32)      0           lstm_21[0][0]                    
__________________________________________________________________________________________________
reshape_6 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`


In [0]:
inputs = np.array([string_to_int(i, TIME_STEPS, source_vocab) for i in source])
targets = [string_to_int(t, TIME_STEPS, target_vocab) for t in target]
targets = np.array(list(map(lambda x: to_categorical(x, num_classes=len(target_vocab)), targets)))

In [81]:
m_t.fit(
    [inputs], targets, 
    epochs=40, batch_size=1000, 
    validation_split=0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/40
90000/90000 [==============================] - 33s 369us/step - loss: 1.9137 - acc: 0.5054 - val_loss: 2.3592 - val_acc: 0.3893
Epoch 2/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.8940 - acc: 0.5097 - val_loss: 2.3342 - val_acc: 0.3952
Epoch 3/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.8780 - acc: 0.5133 - val_loss: 2.3211 - val_acc: 0.3988
Epoch 4/40
39000/90000 [============>.................] - ETA: 18s - loss: 1.8663 - acc: 0.5160

90000/90000 [==============================] - 33s 368us/step - loss: 1.8623 - acc: 0.5171 - val_loss: 2.3044 - val_acc: 0.4019
Epoch 5/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.8482 - acc: 0.5205 - val_loss: 2.2896 - val_acc: 0.4063
Epoch 6/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.8300 - acc: 0.5245 - val_loss: 2.2770 - val_acc: 0.4086
Epoch 7/40
84000/90000 [===========================>..] - ETA: 2s - loss: 1.8154 - acc: 0.5279

90000/90000 [==============================] - 33s 368us/step - loss: 1.8140 - acc: 0.5283 - val_loss: 2.2589 - val_acc: 0.4140
Epoch 8/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.7988 - acc: 0.5323 - val_loss: 2.2487 - val_acc: 0.4182
Epoch 9/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.7851 - acc: 0.5357 - val_loss: 2.2352 - val_acc: 0.4212
Epoch 10/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.7708 - acc: 0.5389 - val_loss: 2.2404 - val_acc: 0.4227
Epoch 11/40


90000/90000 [==============================] - 33s 368us/step - loss: 1.7560 - acc: 0.5418 - val_loss: 2.2104 - val_acc: 0.4263
Epoch 12/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.7435 - acc: 0.5443 - val_loss: 2.2017 - val_acc: 0.4275
Epoch 13/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.7318 - acc: 0.5468 - val_loss: 2.1939 - val_acc: 0.4303
Epoch 14/40
76000/90000 [========================>.....] - ETA: 4s - loss: 1.7188 - acc: 0.5495

90000/90000 [==============================] - 33s 368us/step - loss: 1.7186 - acc: 0.5496 - val_loss: 2.1841 - val_acc: 0.4318
Epoch 15/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.7076 - acc: 0.5522 - val_loss: 2.1781 - val_acc: 0.4335
Epoch 16/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.6957 - acc: 0.5547 - val_loss: 2.1760 - val_acc: 0.4347
Epoch 17/40
89000/90000 [============================>.] - ETA: 0s - loss: 1.6860 - acc: 0.5569

...

Epoch 27/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.6727 - acc: 0.5600 - val_loss: 2.1489 - val_acc: 0.4425
Epoch 28/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.6183 - acc: 0.5701 - val_loss: 2.1543 - val_acc: 0.4441
Epoch 29/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.5998 - acc: 0.5732 - val_loss: 2.1392 - val_acc: 0.4456
Epoch 30/40
68000/90000 [=====================>........] - ETA: 7s - loss: 1.5907 - acc: 0.5748

90000/90000 [==============================] - 33s 367us/step - loss: 1.5899 - acc: 0.5751 - val_loss: 2.1366 - val_acc: 0.4459
Epoch 31/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.5814 - acc: 0.5765 - val_loss: 2.1389 - val_acc: 0.4457
Epoch 32/40
90000/90000 [==============================] - 33s 367us/step - loss: 1.5730 - acc: 0.5781 - val_loss: 2.1379 - val_acc: 0.4472
Epoch 33/40
87000/90000 [============================>.] - ETA: 1s - loss: 1.5655 - acc: 0.5796

90000/90000 [==============================] - 33s 368us/step - loss: 1.5656 - acc: 0.5795 - val_loss: 2.1456 - val_acc: 0.4466
Epoch 34/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5591 - acc: 0.5805 - val_loss: 2.1471 - val_acc: 0.4477
Epoch 35/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5507 - acc: 0.5821 - val_loss: 2.1497 - val_acc: 0.4449
Epoch 36/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5445 - acc: 0.5832 - val_loss: 2.1386 - val_acc: 0.4486


Epoch 37/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5382 - acc: 0.5844 - val_loss: 2.1489 - val_acc: 0.4481
Epoch 38/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5315 - acc: 0.5854 - val_loss: 2.1511 - val_acc: 0.4461
Epoch 39/40
90000/90000 [==============================] - 33s 368us/step - loss: 1.5258 - acc: 0.5868 - val_loss: 2.1579 - val_acc: 0.4490
Epoch 40/40
68000/90000 [=====================>........] - ETA: 7s - loss: 1.5166 - acc: 0.5880

90000/90000 [==============================] - 33s 368us/step - loss: 1.5189 - acc: 0.5877 - val_loss: 2.1518 - val_acc: 0.4451


In [0]:
m.evaluate([X_test], y_test)

In [0]:
run_example(m_t, source_vocab, inv_target_vocab, 'hello')

### Tatoeba Report

* final architectures
* comparison
* as well as training method and tricks
